In [ ]:
!pip3  install selenium
!pip3 install webdriver_manager
!pip3 install lxml
!pip3 install requests
!pip3 install beautifulsoup4

In [69]:
# importing all required library
import pandas as pd

import selenium
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import requests

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


In [55]:
#creating empty list
ET = { }
ET['Date']=[]
ET['Vertical']=[]
ET['Author']=[]
ET['Headline']=[]
ET['Description']=[]
ET['Author1']=[]
ET_URLs=[]


#accessing the webdriver
driver.get('https://economictimes.indiatimes.com/archive.cms?from=mdr')
time.sleep(2)

#selecting the  april month
april = driver.find_element_by_xpath("//*[@id='pageContent']/div[2]/table/tbody/tr[1]/td/span/div/strong/a[264]") 
april.click()


#geeting only ten day url links 
count =5    #Months starts from fifthday(friday)
row =2     

for i in range(30):
    try:
        #Accessing the calender days
        cal2 = driver.find_element_by_xpath(f"/html/body/main/section/div[2]/table/tbody/tr[1]/td/span/div/table/tbody/tr[{row}]/td[{count}]/a")
        cal2.click()
    except NoSuchElementException:
        row +=1
        count=1
        cal3 = driver.find_element_by_xpath(f"/html/body/main/section/div[2]/table/tbody/tr[1]/td/span/div/table/tbody/tr[{row}]/td[{count}]/a")
        cal3.click()
        
    time.sleep(3)
   
        
    #scarping the News URL links 
    for u in driver.find_elements_by_xpath("//*[@id='pageContent']/span/table/tbody/tr/td/ul/li/a"):
        ET_URLs.append(u.get_attribute('href'))
        
    #increamenting the date by adding 1
    count +=1
    
    #going back to calender page 
    try:
        back_april =driver.find_element_by_xpath("//*[@id='pageContent']/span/table[1]/tbody/tr[2]/td/a[3]/b")
        back_april.click()
    except NoSuchElementException:
        driver.get("https://economictimes.indiatimes.com/archive/year-2021,month-4.cms")
        
    time.sleep(3)
    
    
    
    
#SCRAPING THE NEWS DATA
    
for i in ET_URLs:
    try:
        driver.get(i)
        time.sleep(3)
    except TimeoutException:
        driver.get(i)
        time.sleep(3)
        
    try:
        Date_Time = driver.find_element_by_tag_name("time").text
        ET['Date'].append(Date_Time)
    except NoSuchElementException:
        ET['Date'].append('-')
        
    try:
        vertical = driver.find_element_by_xpath("/html/body/main/div[7]/div/span[3]/a").text
        ET['Vertical'].append(vertical)
    except NoSuchElementException:
        ET['Vertical'].append('-')
        
    try:
        Auth = driver.find_element_by_xpath("//div[@class='auth eventDone']/a").text
        ET['Author'].append(Auth)
        
    except NoSuchElementException:
        ET['Author'].append('-')
        
    try:
        Auth1 = driver.find_element_by_xpath("/html/body/main/div[10]/div/div/div[2]/div/div[1]/span[1]").text
        ET['Author1'].append(Auth1)
        
    except NoSuchElementException:
        ET['Author1'].append('-')    
        
    try:
        heading = driver.find_element_by_tag_name("h1").text
        ET['Headline'].append(heading)
    except NoSuchElementException:
        ET['Headline'].append('-')
        
        
    try:
        News_body= driver.find_element_by_xpath("//div[@class='artText medium']").text
        ET['Description'].append(News_body)
    except NoSuchElementException:
        ET['Description'].append('-')
        
    
        


In [ ]:
print(len(ET['Date']),
len(ET['Vertical']),
len(ET['Author']),
len(ET['Headline']),
len(ET['Description']))

In [63]:
# merging the all scarped data  here 
import pandas as pd
import glob

path = r'/home/adarsh/Downloads' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [67]:
len(frame)

10706

In [68]:
EOI =frame.to_csv('EOI_April_news_data.csv')